## **Module**

In [1]:
from wmbio import * 

/home/wmbio/anaconda3/envs/multiomics-cpu/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


## **Best Sub-Group Selection**

* **FILE PATH**

In [2]:
os.chdir("..")
os.getcwd()

'/home/wmbio/WORK/gitworking/Multi-omics-intergration'

In [3]:
# PATH
CANCER_TYPE = "LIHC"
GROUP_PHTH = os.getcwd() + '/group/'
PNG_PATH = os.getcwd() + '/png/'
GROUP_VALIDATION_PATH = os.getcwd() + '/group_validation/'
DEG_PATH = os.getcwd() + "/best_deg/"
RDATA_PATH = os.getcwd() + "/RAW_DATA/GDC_PREPARE/"
METHOD = 'deseq2'

In [4]:
# Load Validation score
col=['FILENAME','Log Rank Test','Silhouette','RNA_ANOVA_F1','RNA_RF_F1',
     'miRNA_ANOVA_F1','miRNA_RF_F1','Methylation_ANOVA_F1','Methylation_RF_F1']

group_score = pd.read_csv(GROUP_VALIDATION_PATH + 'test_validation.csv', usecols=col)

# Condition for Filtering
filter_cond = (group_score['Silhouette'] >= 0.2) & (group_score['Log Rank Test'] < 0.05) & \
              ((group_score['RNA_ANOVA_F1'] > 80) | (group_score['RNA_RF_F1'] > 80)) & \
              ((group_score['miRNA_ANOVA_F1'] > 80) | (group_score['miRNA_RF_F1'] > 80)) & \
              ((group_score['Methylation_ANOVA_F1'] > 85) | (group_score['Methylation_RF_F1'] > 85))
group_score = group_score[filter_cond].sort_values(["Silhouette"], ascending = (False))

bestSubgroup = group_score.FILENAME.to_list()

# 5 fold로 제한.
if len(bestSubgroup) > 10:
    bestSubgroup = bestSubgroup[:10]

# DEA
dea_result = list()
for best_group in bestSubgroup:
    SAMPLE_GROUP = GROUP_PHTH + CANCER_TYPE + "_GROUP_" + best_group + ".txt"
    
    # DEG Extraction
    deg_list = deg_extract(log_fc=1.5, fdr=0.05,
                   cancer_type=CANCER_TYPE, 
                   sample_group=SAMPLE_GROUP, deg_path=DEG_PATH, 
                   file_name=best_group,
                   rdata_path=RDATA_PATH,
                   method=METHOD)
    
    dea_result.append(deg_list)

# Filter DEA
# combine result
if METHOD == 'all':
    dea_combine = list(map(deseq2_edger_combine, dea_result))
    dea_combine = [col_rename(dea_combine[index], index) for index in range(len(dea_combine))]
    dea_combine = reduce(lambda left, right : pd.merge(left, right, left_on='gene', right_on='gene', how = 'outer'), dea_combine)
elif METHOD == 'deseq2' :
    dea_combine = list(map(lambda d : d[["row", "log2FoldChange"]], dea_result))
    dea_combine = [col_rename(dea_combine[index], index) for index in range(len(dea_combine))]
    dea_combine = reduce(lambda left, right : pd.merge(left, right, left_on='gene', right_on='gene', how = 'inner'), dea_combine)

# median & mean
dea_combine["log2FoldChange_median"] = dea_combine.iloc[:, 1:].median(axis=1)
dea_combine["log2FoldChange_mean"] = dea_combine.iloc[:, 1:].mean(axis=1)

In [15]:
group_score

,FILENAME,Log Rank Test,Silhouette,RNA_ANOVA_F1,RNA_RF_F1,miRNA_ANOVA_F1,miRNA_RF_F1,Methylation_ANOVA_F1,Methylation_RF_F1
802,20220203-152500,0.001436,0.2447,89.485736,88.808980,84.117868,83.276256,92.394895,94.617580
2400,20220125-002321,0.031152,0.2435,88.246997,87.427702,83.810060,81.474505,89.237988,91.853120
2647,20220124-145618,0.003107,0.2356,88.412162,87.570396,81.081081,78.046043,86.208709,89.088661
1009,20220202-091554,0.028310,0.2354,89.793544,90.043760,82.338589,82.444825,92.680180,94.339802
1561,20220128-143511,0.022435,0.2350,90.604354,91.712329,82.454955,80.821918,92.676426,92.952816
...,...,...,...,...,...,...,...,...,...
2830,20220121-173830,0.000001,0.2003,84.515766,85.913242,81.212462,81.208143,90.337838,91.031202
918,20220203-071338,0.000002,0.2001,85.900901,86.318493,82.177177,81.759893,91.163664,93.923135
2365,20220125-013942,0.000013,0.2000,85.792042,85.911339,81.332583,79.149543,89.928679,91.291857
2732,20220124-101120,0.000016,0.2000,85.232733,86.063546,81.062312,79.834475,87.860360,91.299467
